In [2]:
%reload_ext autoreload
%autoreload 2


In [3]:
from peptdeep.model.generic_property_prediction import (
    ModelInterface_for_Generic_AASeq_BinaryClassification,
    ModelInterface_for_Generic_AASeq_Regression,
    ModelInterface_for_Generic_ModAASeq_BinaryClassification,
    ModelInterface_for_Generic_ModAASeq_Regression,
)
from peptdeep.model.generic_property_prediction import (
    Model_for_Generic_AASeq_BinaryClassification_LSTM,
    Model_for_Generic_AASeq_BinaryClassification_Transformer,
    Model_for_Generic_AASeq_Regression_LSTM,
    Model_for_Generic_AASeq_Regression_Transformer,
    Model_for_Generic_ModAASeq_BinaryClassification_LSTM,
    Model_for_Generic_ModAASeq_BinaryClassification_Transformer,
    Model_for_Generic_ModAASeq_Regression_LSTM,
    Model_for_Generic_ModAASeq_Regression_Transformer,
)

/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-22 15:18:00.461156: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 15:18:01.224216: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2023-09-22 15:18:11> /cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/peptdeep/mass_spec/ms_reader.py#L12: Cannot import `RawFileReader`, check if PythonNet is installed. See https://github.com/MannLabs/alphapeptdeep#pip


In [4]:
model = ModelInterface_for_Generic_ModAASeq_Regression(
    model_class=Model_for_Generic_ModAASeq_Regression_Transformer
)

In [6]:
vars(model)

{'model': Model_for_Generic_ModAASeq_Regression_Transformer(
   (dropout): Dropout(p=0.1, inplace=False)
   (input_nn): AA_Mod_Embedding(
     (mod_embedding): Mod_Embedding_FixFirstK(
       (nn): Linear(in_features=103, out_features=2, bias=False)
     )
     (aa_embedding): Embedding(128, 248, padding_idx=0)
   )
   (hidden_nn): HFace_Transformer_with_PositionalEncoder(
     (pos_encoder): PositionalEncoding()
     (bert): Hidden_HFace_Transformer(
       (bert): BertEncoder(
         (layer): ModuleList(
           (0): BertLayer(
             (attention): BertAttention(
               (self): BertSelfAttention(
                 (query): Linear(in_features=256, out_features=256, bias=True)
                 (key): Linear(in_features=256, out_features=256, bias=True)
                 (value): Linear(in_features=256, out_features=256, bias=True)
                 (dropout): Dropout(p=0.1, inplace=False)
               )
               (output): BertSelfOutput(
                 (dense):

In [7]:
import torch.nn as nn

In [16]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

AttributeError: 'ModelInterface_for_Generic_ModAASeq_Regression' object has no attribute 'parameters'

In [17]:
from peptdeep.model.ms2 import ModelMS2Transformer


In [18]:
ms2 = ModelMS2Transformer(num_frag_types)

NameError: name 'num_frag_types' is not defined

In [38]:
import torch
class SeqAttentionSum(torch.nn.Module):
    """
    apply linear transformation and tensor rescaling with softmax
    """
    def __init__(self, in_features):
        super().__init__()
        self.attn = torch.nn.Sequential(
            torch.nn.Linear(in_features, 1, bias=False),
            torch.nn.Softmax(dim=1),
        )
    
    def forward(self, x):
        print(x)
        attn = self.attn(x)
        print(attn)
        print(torch.sum(torch.mul(x, attn), dim=1))
        return torch.sum(torch.mul(x, attn), dim=1)

In [39]:
t = SeqAttentionSum(50)

In [44]:
import numpy as np
in_ =  torch.rand(2,50)
t(in_).shape

tensor([[9.2608e-01, 1.5943e-01, 2.5331e-01, 9.8373e-01, 7.4680e-01, 9.7373e-01,
         5.6774e-01, 2.8373e-01, 4.6799e-01, 5.4813e-01, 4.0378e-01, 6.4741e-01,
         1.3489e-01, 2.4661e-01, 7.7137e-01, 4.8373e-01, 4.7676e-01, 2.2019e-01,
         7.8809e-02, 2.4353e-01, 6.7434e-01, 6.0718e-01, 3.4206e-01, 5.2088e-01,
         2.4333e-01, 5.8480e-01, 4.7504e-02, 5.7012e-01, 3.1573e-01, 3.8020e-01,
         7.6796e-01, 5.0488e-01, 2.2465e-01, 1.1290e-01, 3.2906e-01, 8.6623e-01,
         3.1009e-02, 5.0773e-01, 9.1490e-01, 3.1279e-01, 1.2669e-01, 4.8335e-01,
         6.6977e-01, 3.1997e-01, 9.9042e-01, 4.7055e-01, 4.0654e-01, 7.1775e-02,
         4.2031e-01, 7.7907e-01],
        [2.6743e-01, 3.3529e-02, 4.6635e-01, 7.9669e-01, 2.1487e-01, 3.6744e-01,
         9.4360e-01, 7.2820e-01, 8.0573e-01, 5.2109e-01, 7.7740e-01, 7.6125e-01,
         9.5645e-01, 2.4470e-01, 4.2113e-01, 3.6812e-01, 9.6741e-01, 6.0959e-01,
         9.1718e-01, 8.8984e-01, 3.3928e-01, 8.4326e-01, 2.0324e-01, 2.0622

torch.Size([2])

In [43]:
torch.sum(in_, dim=1)

tensor([21.7500, 24.2620])